In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 48.5 MB/s eta 0:00:00


In [ ]:
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.svm import SVR
import numpy as np


In [ ]:

import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, rdMolDescriptors, MACCSkeys
from rdkit.Avalon import pyAvalonTools
from rdkit.Chem.rdFingerprintGenerator import (
    GetMorganGenerator,
    GetAtomPairGenerator,
    GetTopologicalTorsionGenerator,
)
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# ---------------------------
# Load datasets once
# ---------------------------
df = pd.read_excel('HOMO-LUMO-energies.xlsx')
smiles_list = df["Smiles"]
y = df["dFF"].values
homo = df["HOMO energy (eV)"].values
lumo = df["LUMO energy (eV)"].values
mols = [Chem.MolFromSmiles(s) for s in smiles_list]

df_new = pd.read_csv('INPUT-NEW-MOLS-correct-HOMOLUMO.csv')
smiles_new = df_new["Smiles"]
homo_new = df_new["HOMO (eV)"].values
lumo_new = df_new["LUMO (eV)"].values
mols_new = [Chem.MolFromSmiles(s) for s in smiles_new]

# (Optional) filter out any None molecules if present
def _filter_none(ms, *arrs):
    idx = [i for i,m in enumerate(ms) if m is not None]
    ms = [ms[i] for i in idx]
    arrs = [np.asarray(a)[idx] for a in arrs]
    return (ms, *arrs)

mols, y, homo, lumo = _filter_none(mols, y, homo, lumo)
mols_new, homo_new, lumo_new = _filter_none(mols_new, homo_new, lumo_new)

# ---------------------------
# Fingerprint builders (bit vectors) — generator API (no deprecation warnings)
# ---------------------------
# Morgan (ECFP2): radius=2, 2048 bits, binary (no count simulation)
_morgan_gen = GetMorganGenerator(radius=2, fpSize=2048, countSimulation=False)
def fp_morgan(ms):
    return [_morgan_gen.GetFingerprint(m) for m in ms]

# MACCS (no generator API needed)
def fp_maccs(ms):
    return [MACCSkeys.GenMACCSKeys(m) for m in ms]

# Avalon
def fp_avalon(ms):
    return [pyAvalonTools.GetAvalonFP(m) for m in ms]

# Daylight/RDKit FP (legacy call is still fine)
def fp_daylight(ms):
    return [Chem.RDKFingerprint(m) for m in ms]

# AtomPairs via generator (binary)
_atompair_gen = GetAtomPairGenerator(fpSize=2048)
def fp_atompairs(ms):
    return [_atompair_gen.GetFingerprint(m) for m in ms]

# Topological Torsion via generator (binary)
_torsion_gen = GetTopologicalTorsionGenerator(fpSize=2048)
def fp_torsion(ms):
    return [_torsion_gen.GetFingerprint(m) for m in ms]

# ---------------------------
# Helper: stack fingerprint bits with HOMO/LUMO features
# ---------------------------
def assemble_features(bitvecs, homo_vals, lumo_vals):
    # ExplicitBitVect from generators is iterable => list of 0/1 bits
    return np.array([[*bv, h, l] for bv, h, l in zip(bitvecs, homo_vals, lumo_vals)], dtype=float)

# ---------------------------
# Core training routine (keeps top-5 by R² across 200 splits)
# ---------------------------
def run_svr_combo(fp_name, kernel, fp_func):
    X = assemble_features(fp_func(mols), homo, lumo)
    X_new = assemble_features(fp_func(mols_new), homo_new, lumo_new)

    tag = f"{kernel}-{fp_name}".lower()

    best_models = []
    best_scores = [-np.inf]*5

    with open(f"out-SVR-{tag}.txt", "w") as f:
        for i in range(200):
            Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.20, random_state=i)
            model = SVR(kernel=kernel)
            model.fit(Xtr, ytr)
            ypred = model.predict(Xte)
            r2v = r2_score(yte, ypred)

            # maintain top-5 list
            for j, s in enumerate(best_scores):
                if r2v > s:
                    best_models.insert(j, model)
                    best_scores.insert(j, r2v)
                    del best_models[5:]
                    del best_scores[5:]
                    break

            print(f"Iteration {i+1}: R2 Score = {r2v}", file=f)

    # Predict new set with each of the best models
    preds_list = [m.predict(X_new) for m in best_models]
    pred_df = pd.concat([pd.DataFrame(p, columns=[f"Model_{k+1}_Prediction"])
                         for k, p in enumerate(preds_list)], axis=1)
    pred_df.T.to_csv(f"predicted_dff-{tag}-en.csv", header=False)

    with open(f"r2_values_best-{tag}-en.txt", "w") as f:
        for k, s in enumerate(best_scores):
            f.write(f"Best Model {k+1} R2: {s}\n")

# ---------------------------
# Run the 11 requested combinations
# ---------------------------
# Morgan – RBF
run_svr_combo("morgan", "rbf", fp_morgan)

# MACCS – RBF
run_svr_combo("maccs", "rbf", fp_maccs)

# Avalon – Linear, Sigmoid
run_svr_combo("avalon", "linear", fp_avalon)
run_svr_combo("avalon", "sigmoid", fp_avalon)

# Daylight – Linear, Sigmoid
run_svr_combo("daylight", "linear", fp_daylight)
run_svr_combo("daylight", "sigmoid", fp_daylight)

# AtomPairs – Linear, RBF, Sigmoid
run_svr_combo("atompairs", "linear", fp_atompairs)
run_svr_combo("atompairs", "rbf", fp_atompairs)
run_svr_combo("atompairs", "sigmoid", fp_atompairs)

# Torsion – Linear, RBF
run_svr_combo("torsion", "linear", fp_torsion)
run_svr_combo("torsion", "rbf", fp_torsion)


# only fingerprints

In [ ]:
# -*- coding: utf-8 -*-
"""
SVR with FINGERPRINTS ONLY (no HOMO/LUMO)
- Trains 200 random train/test splits for each fingerprint–kernel combo
- Keeps Top-5 models by test R^2 for each combo
- Predicts blind set with each kept model
- Saves per-combo predictions and scores
- Computes GLOBAL ENSEMBLE across ALL kept models from ALL combos
- Additionally saves MEAN and STD of R^2 (across 200 runs) for each combo

Inputs expected:
 1) TRAIN file (Excel): must contain columns 'Smiles' and 'dFF'
 2) BLIND file (CSV): must contain column 'Smiles'

Outputs written:
  - out-SVR-<kernel>-<fp>.txt               (log of all 200 runs)
  - r2_values_best-<kernel>-<fp>-fp.txt     (Top-5 R^2 values)
  - predicted_dff-<kernel>-<fp>-fp.csv      (Blind predictions from Top-5 models for that combo)
  - ensemble_predictions_all_models.csv      (GLOBAL ensemble across ALL kept models)
  - combo_summaries.json                    (summary with mean, std R^2, top R^2 per combo)
"""

from pathlib import Path
import json
import numpy as np
import pandas as pd

from rdkit import RDLogger
RDLogger.DisableLog("rdApp.*")

from rdkit import Chem, DataStructs
from rdkit.Chem import rdMolDescriptors, MACCSkeys
from rdkit.Chem.rdMolDescriptors import GetHashedTopologicalTorsionFingerprintAsBitVect, GetHashedAtomPairFingerprintAsBitVect
from rdkit.Avalon.pyAvalonTools import GetAvalonFP

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

TRAIN_XLSX = "HOMO-LUMO-energies.xlsx"
BLIND_CSV  = "INPUT-NEW-MOLS-correct-HOMOLUMO.csv"
OUTDIR     = Path("./outputs_fp_only")
OUTDIR.mkdir(parents=True, exist_ok=True)

def mols_from_smiles(smiles_list):
    mols, keep_smiles = [], []
    for s in smiles_list:
        m = Chem.MolFromSmiles(str(s))
        if m is not None:
            mols.append(m)
            keep_smiles.append(str(s))
    return mols, keep_smiles

def bitvect_to_array(fp):
    arr = np.zeros((1,), dtype=int)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

def fp_morgan(mols, radius=2, nBits=2048):
    fps = [rdMolDescriptors.GetMorganFingerprintAsBitVect(m, radius, nBits=nBits) for m in mols]
    return np.vstack([bitvect_to_array(fp) for fp in fps])

def fp_maccs(mols):
    fps = [MACCSkeys.GenMACCSKeys(m) for m in mols]
    return np.vstack([bitvect_to_array(fp) for fp in fps])

def fp_avalon(mols, nBits=1024):
    fps = [GetAvalonFP(m, nBits=nBits) for m in mols]
    return np.vstack([bitvect_to_array(fp) for fp in fps])

def fp_daylight(mols, nBits=2048):
    fps = [Chem.RDKFingerprint(m, fpSize=nBits) for m in mols]
    return np.vstack([bitvect_to_array(fp) for fp in fps])

def fp_atompairs(mols, nBits=2048):
    fps = [GetHashedAtomPairFingerprintAsBitVect(m, nBits=nBits) for m in mols]
    return np.vstack([bitvect_to_array(fp) for fp in fps])

def fp_torsion(mols, nBits=2048):
    fps = [GetHashedTopologicalTorsionFingerprintAsBitVect(m, nBits=nBits) for m in mols]
    return np.vstack([bitvect_to_array(fp) for fp in fps])

FINGERPRINTS = {
    "morgan":   fp_morgan,
    "maccs":    fp_maccs,
    "avalon":   fp_avalon,
    "daylight": fp_daylight,
    "atompairs":fp_atompairs,
    "torsion":  fp_torsion,
}

COMBOS = {
    "morgan":   ["rbf"],
    "maccs":    ["rbf"],
    "avalon":   ["linear", "sigmoid"],
    "daylight": ["linear", "sigmoid"],
    "atompairs":["linear", "rbf", "sigmoid"],
    "torsion":  ["linear", "rbf"],
}

train_df = pd.read_excel(TRAIN_XLSX)
blind_df = pd.read_csv(BLIND_CSV)

train_mols, train_smiles = mols_from_smiles(train_df["Smiles"].tolist())
blind_mols, blind_smiles = mols_from_smiles(blind_df["Smiles"].tolist())

kept_train_df = train_df[train_df["Smiles"].astype(str).isin(train_smiles)].reset_index(drop=True)
y = kept_train_df["dFF"].values.astype(float)

def run_svr_combo(fp_name, kernel, fp_func, n_iter=200, test_size=0.2):
    X = fp_func(train_mols)
    X_new = fp_func(blind_mols)

    log_lines, kept, all_r2s = [], [], []

    for i in range(n_iter):
        X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=test_size, random_state=i)
        model = SVR(kernel=kernel)
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        r2 = r2_score(y_te, y_pred)
        all_r2s.append(r2)

        log_lines.append(f"iter={i:03d} R2={r2:.5f}")

        if len(kept) < 5:
            kept.append((r2, model))
            kept.sort(key=lambda t: t[0], reverse=True)
        else:
            if r2 > kept[-1][0]:
                kept[-1] = (r2, model)
                kept.sort(key=lambda t: t[0], reverse=True)

    log_path = OUTDIR / f"out-SVR-{kernel}-{fp_name}.txt"
    with open(log_path, "w", encoding="utf-8") as f:
        f.write("\n".join(log_lines))

    r2s = [t[0] for t in kept]
    with open(OUTDIR / f"r2_values_best-{kernel}-{fp_name}-fp.txt", "w", encoding="utf-8") as f:
        for r in r2s:
            f.write(f"{r:.6f}\n")

    preds = []
    for (_, mdl) in kept:
        preds.append(mdl.predict(X_new))

    combo_df = pd.DataFrame({
        "Smiles": blind_smiles,
        **{f"pred_model{j+1}": preds[j] for j in range(len(preds))}
    })
    combo_df.to_csv(OUTDIR / f"predicted_dff-{kernel}-{fp_name}-fp.csv", index=False)

    mean_r2 = float(np.mean(all_r2s))
    std_r2 = float(np.std(all_r2s, ddof=0))
    return kept, np.vstack(preds), mean_r2, std_r2

all_preds, combo_summaries = [], []

for fp_name, kernels in COMBOS.items():
    fp_func = FINGERPRINTS[fp_name]
    for kernel in kernels:
        print(f"Running {fp_name} + {kernel} ...")
        kept_models, combo_preds, mean_r2, std_r2 = run_svr_combo(fp_name, kernel, fp_func)
        for k in range(combo_preds.shape[0]):
            all_preds.append(combo_preds[k, :])
        combo_summaries.append({
            "fingerprint": fp_name,
            "kernel": kernel,
            "top5_r2_mean": float(np.mean([r for r, _ in kept_models])),
            "top_r2": float(max([r for r, _ in kept_models])),
            "mean_r2_all_runs": mean_r2,
            "std_r2_all_runs": std_r2,
        })

all_preds = np.vstack(all_preds)
ens_mean = np.mean(all_preds, axis=0)
ens_median = np.median(all_preds, axis=0)
ens_std = np.std(all_preds, axis=0, ddof=0)

ensemble_df = pd.DataFrame({
    "Smiles": blind_smiles,
    "ensemble_mean": ens_mean,
    "ensemble_median": ens_median,
    "ensemble_std": ens_std,
    "n_models": all_preds.shape[0],
})

ensemble_df.to_csv(OUTDIR / "ensemble_predictions_all_models.csv", index=False)

with open(OUTDIR / "combo_summaries.json", "w", encoding="utf-8") as f:
    json.dump(combo_summaries, f, indent=2)

print("\n=== GLOBAL ENSEMBLE (preview) ===")
print(ensemble_df.head(10).to_string(index=False))
print("\nPer-combo mean & std R2s:")
for c in combo_summaries:
    print(f"{c['fingerprint']} + {c['kernel']}: mean_r2_all_runs={c['mean_r2_all_runs']:.4f}, std_r2_all_runs={c['std_r2_all_runs']:.4f}")
print(f"\nSaved outputs to {OUTDIR}")



Running morgan + rbf ...
Running maccs + rbf ...
Running avalon + linear ...
Running avalon + sigmoid ...
Running daylight + linear ...
Running daylight + sigmoid ...
Running atompairs + linear ...
Running atompairs + rbf ...
Running atompairs + sigmoid ...
Running torsion + linear ...
Running torsion + rbf ...

=== GLOBAL ENSEMBLE (preview) ===
                             Smiles  ensemble_mean  ensemble_median  ensemble_std  n_models
              CC1=C(O)C(O)=C(C)C=C1       0.568112         0.496263      0.225037        55
         OC1=CC=C(C=C1O)C1=CC=CC=C1       0.707005         0.690736      0.185717        55
         [NH3+]CCCC1=CC(O)=C(O)C=C1       0.576819         0.558810      0.253336        55
           [NH3+]CC1=CC=C(O)C(O)=C1       0.677487         0.670461      0.146004        55
                  CNC1=C(NC)C=CC=C1       0.321066         0.196121      0.382898        55
            CC(C)(C)C1=CC=CC(O)=C1O       0.460660         0.443018      0.221100        55
        